<a href="https://colab.research.google.com/github/sheon-j/nlp-study/blob/main/tensorflow-nlp-tutorial/week06_word_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Study: Week 6 - Word Embedding

[딥 러닝을 이용한 자연어 처리 입문](https://wikidocs.net/book/2155) 스터디

---

**Contents**
8. 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)
9. 엘모(Embeddings from Language Model, ELMo)
10. 임베딩 벡터의 시각화(Embedding Visualization)

## 8. 사전 훈련된 워드 임베딩(Pre-trained Word Embedding)

### 8.1. 케라스 임베딩 층(Keras Embedding layer)

### 8.2. 사전 훈련된 워드 임베딩(Pre-Trained Word Embedding) 사용하기

## 9. 엘모(Embeddings from Language Model, ELMo)

### 9.1. ELMo(Embeddings from Language Model)

### 9.2. biLM(Bidirectional Language Model)의 사전 훈련

### 9.3. biLM의 활용

### 9.4. ELMo 표현을 사용해서 스팸 메일 분류하기

## 10. 임베딩 벡터의 시각화(Embedding Visualization)

### 10.1. 워드 임베딩 모델로부터 2개의 tsv 파일 생성하기

### 10.2. 임베딩 프로젝터를 사용하여 시각화하기